In [1]:
!pip install pyarrow
!pip install fastparquet

### Overview

In this notebook, I would like to present a framework for **partitioned validation** using one-week hold out.  
As an example of possible groupings of customers, here I choose '**online vs. offline**', which shows that it is more difficult to predict future purchases of offline uses than to predict those of online users.  
  
I use some tequniqus I learned from great discussions and notebooks shared in the H&M competition.  
So, I would really like to thank all the authors of them.

- **Version 6**  
I added and fixed the code so as to make it easier to record and show results of validations on the week from 2020-9-16 to 2020-9-22.  
In Version 6, MAP@12 of hold-out validation is around 0.0183 for online users and 0.0298 for offline users.  

### Libraries and Functions

In [2]:
import numpy as np, pandas as pd, datetime as dt
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
from collections import defaultdict

def iter_to_str(iterable):
    return " ".join(map(lambda x: str(0) + str(x), iterable))

def apk(actual, predicted, k=12):
    if len(predicted) > k:
        predicted = predicted[:k]
    score, nhits = 0.0, 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            nhits += 1.0
            score += nhits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=12, return_apks=False):
    assert len(actual) == len(predicted)
    apks = [apk(ac, pr, k) for ac, pr in zip(actual, predicted) if 0 < len(ac)]
    if return_apks:
        return apks
    return np.mean(apks)

def blend(dt, w=[], k=12):
    if len(w) == 0:
        w = [1] * (len(dt))
    preds = []
    for i in range(len(w)):
        preds.append(dt[i].split())
    res = {}
    for i in range(len(preds)):
        if w[i] < 0:
            continue
        for n, v in enumerate(preds[i]):
            if v in res:
                res[v] += (w[i] / (n + 1))
            else:
                res[v] = (w[i] / (n + 1))    
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    return ' '.join(res[:k])

def prune(pred, ok_set, k=12):
    pred = pred.split()
    post = []
    for item in pred:
        if int(item) in ok_set and not item in post:
            post.append(item)
    return " ".join(post[:k])

def validation(actual, predicted, grouping, score=0, index=-1, ignore=False, figsize=(12, 6)):
    # actual, predicted : list of lists
    # group : pandas Series
    # score : pandas DataFrame
    if ignore: return
    ap12 = mapk(actual, predicted, return_apks=True)
    map12 = round(np.mean(ap12), 6)
    if isinstance(score, int): score = pd.DataFrame({g:[] for g in sorted(grouping.unique().tolist())})
    if index == -1 : index = score.shape[0]
    score.loc[index, "All"] = map12
    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1); sns.histplot(data=ap12, log_scale=(0, 10), bins=20); plt.title(f"MAP@12 : {map12}")
    for g in grouping.unique():
        map12 = round(mapk(actual[grouping == g], predicted[grouping == g]), 6)
        score.loc[index, g] = map12
    plt.subplot(1, 2, 2); score[[g for g in grouping.unique()[::-1]] + ['All']].loc[index].plot.barh(); plt.title(f"MAP@12 of Groups")
    vc = pd.Series(predicted).apply(len).value_counts()
    score.loc[index, "Fill"] = round(1 - sum(vc[k] * (12 - k) / 12 for k in (set(range(12)) & set(vc.index))) / len(actual), 3) * 100
    display(score)
    return score

### wod2vec

In [3]:
import json
from  gensim import models

model = models.Word2Vec.load('../input/handmitem2vec/word2vec_v2.model')
with open('../input/handmitem2vec/cust_buy.json', "r") as f :
    cust_buy = json.load(f)

### Data

In [4]:
df = pd.read_parquet('../input/hm-parquets-of-datasets/transactions_train.parquet')
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
cid = pd.DataFrame(sub.customer_id.apply(lambda s: int(s[-16:], 16)))

### Definition of Groups

In [5]:
group = df.groupby('customer_id').sales_channel_id.mean().round().reset_index()\
    .merge(cid, on='customer_id', how='right').rename(columns={'sales_channel_id':'group'})
grouping = group.group.fillna(1.0)

### One-Week Hold Out

In [6]:
val_week = 105
# id of week to be used in a validation; set 105 if you would like to create a submission
val = df.loc[df.week == val_week].groupby('customer_id').article_id.apply(iter_to_str).reset_index()\
    .merge(cid, on='customer_id', how='right')
actual = val.article_id.apply(lambda s: [] if pd.isna(s) else s.split())
last_date = df.loc[df.week < val_week].t_dat.max()

In [7]:
last_date

Timestamp('2020-09-22 00:00:00')

### 最後に購入したアイテム(14日前から）

In [8]:
init_date = last_date - dt.timedelta(days=9999)
train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()
train = train.merge(train.groupby('customer_id').t_dat.max().reset_index().rename(columns={'t_dat':'l_dat'}),
                   on = 'customer_id', how='left')
train['d_dat'] = (train.l_dat - train.t_dat).dt.days
train = train.loc[train.d_dat < 14].sort_values(['t_dat'], ascending=False).drop_duplicates(['customer_id', 'article_id'])
sub['last_purchase'] = train.groupby('customer_id')\
    .article_id.apply(iter_to_str).reset_index()\
    .merge(cid, on='customer_id', how='right').article_id.fillna('')

predicted = sub['last_purchase'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, index='Last Purchase', ignore=(val_week == 105))

### 購入したアイテムの他の色（６日前まで）

In [9]:
init_date = last_date - dt.timedelta(days=6)
#init_date = last_date - dt.timedelta(days=14)
train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()\
    .groupby(['article_id']).t_dat.count().reset_index()
adf = pd.read_parquet('../input/hm-parquets-of-datasets/articles.parquet')
adf = adf.merge(train, on='article_id', how='left').rename(columns={'t_dat':'ct'})\
    .sort_values('ct', ascending=False).query('ct > 0')

map_to_col = defaultdict(list)
for aid in adf.article_id.tolist():
    map_to_col[aid] = list(filter(lambda x: x != aid, adf[adf.product_code == aid // 1000].article_id.tolist()))[:1]

def map_to_variation(s):
    f = lambda item: iter_to_str(map_to_col[int(item)])
    return ' '.join(map(f, s.split()))
sub['other_colors'] = sub['last_purchase'].fillna('').apply(map_to_variation)

In [10]:
from tqdm import tqdm


def sort_item2vec(colname, max_n = 2, none = "") :
    l = []
    for i in tqdm(range(len(sub))) :
        wid = sub.iloc[i]['customer_id']
        oc = sub.iloc[i][colname].split()
        if len(oc) == 0 :
            l.append(none)
        elif wid not in cust_buy :
            l.append(" ".join(oc))            
        elif len(oc) <= 1 :
            l.append(" ".join(oc))
        else :
            ll = []
            for e in oc : 
                buy = cust_buy[wid].split(" ")
                p = model.wv.similarity(buy, e).mean()
                ll.append([p, e])
            ll.sort(reverse=True)
            l.append(" ".join(list(map(lambda x: x[1], ll[:max_n]))))
    #     if i == 10: break

    return l

In [11]:
sub['other_colors'] = sort_item2vec('other_colors', max_n=2)

100%|██████████| 1371980/1371980 [07:49<00:00, 2925.02it/s]


In [12]:
predicted = sub['other_colors'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Other Colors', ignore=(val_week == 105))

### 各グループの人気商品（４日前まで）グループは1,2,na<-1

In [13]:
# cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
# cdf.fashion_news_frequency.unique()
# cdf.head()

In [14]:
cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
cdf = cdf[['customer_id','age','attribute']]
#listBin = [-1, 19, 29, 39, 49, 59, 69, 119]
listBin = [-1, 19, 34, 49, 119]
#cdf['cust_feature'] = pd.cut(cdf['age'], listBin).astype(str).str.cat(cdf['attribute'].astype(str))
cdf['cust_feature'] = pd.qcut(cdf['age'], 10).astype(str).str.cat(cdf['attribute'].astype(str))
#cdf['cust_feature'] = pd.cut(cdf['age'], listBin).astype(str)
#cdf['cust_feature'] = pd.qcut(cdf['age'], 10).astype(str)
cdf = cdf.drop('age', axis=1)
cdf = cdf.drop('attribute', axis=1)
cdf['cust_feature'].unique()

array(['(46.0, 51.0]Woman', '(23.0, 25.0]Woman',
       '(51.0, 57.0]Sports-person', '(51.0, 57.0]Woman',
       '(-1.001, 20.0]Sports-person', '(-1.001, 20.0]Young',
       '(31.0, 38.0]Woman', '(-1.001, 20.0]Woman', '(28.0, 31.0]Young',
       '(28.0, 31.0]Have-kids', '(28.0, 31.0]Woman', '(57.0, 99.0]Woman',
       '(38.0, 46.0]Woman', '(25.0, 28.0]Man', '(20.0, 23.0]Woman',
       '(31.0, 38.0]Man', '(38.0, 46.0]Have-kids', '(25.0, 28.0]Woman',
       '(57.0, 99.0]Young', '(20.0, 23.0]Young', '(23.0, 25.0]Young',
       '(46.0, 51.0]Man', '(31.0, 38.0]Young', '(-1.001, 20.0]Man',
       '(25.0, 28.0]Young', '(57.0, 99.0]Man', '(31.0, 38.0]Have-kids',
       '(20.0, 23.0]None', '(38.0, 46.0]Sports-person',
       '(46.0, 51.0]Young', '(46.0, 51.0]Have-kids',
       '(25.0, 28.0]Sports-person', '(38.0, 46.0]Young',
       '(28.0, 31.0]Sports-person', '(57.0, 99.0]Sports-person',
       '(20.0, 23.0]Man', '(23.0, 25.0]Man', '(51.0, 57.0]Man',
       '(51.0, 57.0]Young', '(46.0, 51.0]S

In [15]:
init_date = last_date - dt.timedelta(days=5 - 1)
group_df = pd.concat([cid, group.group.fillna(1)], axis=1) # grouping can be changed
group_df = group_df.merge(cdf, on='customer_id', how='right')
group_df.columns = ['customer_id', 'group', 'cust_feature']
group_df['cust_feature'] = group_df['group'].astype(str).str.cat(group_df['cust_feature'])
train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()\
    .merge(group_df, on='customer_id', how='left')\
    .groupby(['cust_feature', 'article_id']).t_dat.count().reset_index()

items = defaultdict(str)
for g in train.cust_feature.unique():
    items[g] = iter_to_str(train.loc[train.cust_feature == g].sort_values('t_dat', ascending=False).article_id.tolist()[:30])

sub['popular_items'] = group_df.cust_feature.map(items)

In [16]:
normal = "0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001"
sub['popular_items'] = sort_item2vec('popular_items', max_n=12, none=normal)

100%|██████████| 1371980/1371980 [56:31<00:00, 404.48it/s] 


In [17]:
predicted = sub['popular_items'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Popular Items', ignore=(val_week == 105))

In [18]:
sub.to_csv('submission_all.csv', index=False)

In [19]:
import gc
del model
del cust_buy
gc.collect()

30

### 予測をブレンド

In [20]:
# 11日間の売れた商品リスト
init_date = last_date - dt.timedelta(days=11)
sold_set = set(df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].article_id.tolist())

In [21]:
# 11日間に販売実績のある商品だけ対象
sub['prediction'] = sub[['last_purchase', 'other_colors', 'popular_items']]\
    .apply(blend, w=[100, 10, 1], axis=1, k=32).apply(prune, ok_set=sold_set)
predicted = sub.prediction.apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Prediction', ignore=(val_week == 105))

In [22]:
len(sub['prediction'][0].split())
#sns.barplot(data=score, x='All', y=score.index)

12

In [23]:
if val_week == 105: sub[['customer_id', 'prediction']].to_csv('submission.csv', index=False)

In [24]:
sub['prediction'] = sub['popular_items']

In [25]:
if val_week == 105: sub[['customer_id', 'prediction']].to_csv('submission_popular.csv', index=False)